<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Kingston/ZSL-v2/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Kingston/ZSL-v2


In [2]:
!pip install transformers 

In [3]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/Taedriel/ZSL-v2
   8cff4ce..1d32dea  wordEmbedding -> origin/wordEmbedding
Updating 8cff4ce..1d32dea
Fast-forward
 converter.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [4]:
from converter import *

In [5]:
model = BERTModel()

model.importTagList("en-basic")
model.convert()
model.export("en-basic-Embedded.csv")

# model.computeCoSim()
# model.simBetween("cat", "dog")

men = model.getEmbeddingOf("man")[1]
woman = model.getEmbeddingOf("woman")[1]

boy = model.getEmbeddingOf("boy")[1]

totest = men.add(woman).sub(boy)
model.getNearestEmbeddingOf(totest)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Import finished : 850 elements imported.


['I', 'a', 'able', 'about', 'account', 'acid', 'across', 'act', 'addition', 'adjustment', 'advertisement', 'after', 'again', 'against', 'agreement', 'air', 'all', 'almost', 'among', 'amount', 'amusement', 'and', 'angle', 'angry', 'animal', 'answer', 'ant', 'any', 'apparatus', 'apple', 'approval', 'arch', 'argument', 'arm', 'army', 'art', 'as', 'at', 'attack', 'attempt', 'attention', 'attraction', 'authority', 'automatic', 'awake', 'baby', 'back', 'bad', 'bag', 'balance', 'ball', 'band', 'base', 'basin', 'basket', 'bath', 'be', 'beautiful', 'because', 'bed', 'bee', 'before', 'behaviour', 'belief', 'bell', 'bent', 'berry', 'between', 'bird', 'birth', 'bit', 'bite', 'bitter', 'black', 'blade', 'blood', 'blow', 'blue', 'board', 'boat', 'body', 'boiling', 'bone', 'book', 'boot', 'bottle', 'box', 'boy', 'brain', 'brake', 'branch', 'brass', 'bread', 'breath', 'brick', 'bridge', 'bright', 'broken', 'brother', 'brown', 'brush', 'bucket', 'building', 'bulb', 'burn', 'burst', 'business', 'but', '

[('sense', tensor(0.7157)),
 ('room', tensor(0.7164)),
 ('ant', tensor(0.7189)),
 ('collar', tensor(0.7198)),
 ('statement', tensor(0.7234)),
 ('substance', tensor(0.7270)),
 ('frame', tensor(0.7314)),
 ('person', tensor(0.7565)),
 ('man', tensor(0.8709)),
 ('woman', tensor(0.8795))]